<a href="https://colab.research.google.com/github/Mingjia000/Master_thesis/blob/main/PPO_mode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import time
from datetime import datetime

import torch
import torch.nn as nn
from torch.distributions import MultivariateNormal
from torch.distributions import Categorical
import numpy as np

In [18]:

################################## set device ##################################

print("============================================================================================")


# set device to cpu or cuda
device = torch.device('cpu')

if(torch.cuda.is_available()): 
    device = torch.device('cuda:0') 
    torch.cuda.empty_cache()
    print("Device set to : " + str(torch.cuda.get_device_name(device)))
else:
    print("Device set to : cpu")
    
print("============================================================================================")

class RolloutBuffer:
    def __init__(self):
        self.actions = []
        self.states = []
        self.logprobs = []
        self.rewards = []
        self.is_terminals = []

    def clear(self):
        del self.actions[:]
        del self.states[:]
        del self.logprobs[:]
        del self.rewards[:]
        del self.is_terminals[:]

class ActorCritic(nn.Module):
    def __init__(self, state_dim, action_dim, action_std_init):
        super(ActorCritic, self).__init__()

        self.actor = nn.Sequential(
                            nn.Linear(state_dim, 64),
                            nn.ReLU(),
                            nn.Linear(64, 64),
                            nn.ReLU(),
                            nn.Linear(64, action_dim),
                            nn.Softmax(dim=-1)
                        )

        self.critic = nn.Sequential(
                            nn.Linear(state_dim, 64),
                            nn.ReLU(),
                            nn.Linear(64, 64),
                            nn.ReLU(),
                            nn.Linear(64, 1)
                    )
    def forward(self):
        raise NotImplementedError   

    def act(self, state):

        action_probs = self.actor(state)
        dist = Categorical(action_probs)
        #print(action_probs)

        action = dist.sample()
        #print('sample result',action)

        action_logprob = dist.log_prob(action)
        
        return action.detach(), action_logprob.detach()    

        '''
        action_probs = self.actor(state)
        action_probs_mode = action_probs[0:3] # 3 modes are considered
        action_probs_route = action_probs[3:]

        dist_mode = Categorical(action_probs_mode)
        dist_route = Categorical(action_probs_route)
        #print(action_probs)

        action_mode = dist_mode.sample()
        action_route = dist_route.sample()
        action=torch.cat((action_mode, action_route))
        #print('sample result',action)

        #action_logprob=dist.log_prob(action)
        action_logprob=torch.log(dist_mode[action_mode]*dist_route[action_route])
         '''        
        return action.detach(), action_logprob.detach()      
   
    def evaluate(self, state, action):

        state = state.view(-1, 1)

        action_probs = self.actor(state)
        dist = Categorical(action_probs)

        action_logprobs = dist.log_prob(action)
        dist_entropy = dist.entropy()
        state_values = self.critic(state)
        
        return action_logprobs, state_values, dist_entropy

class PPO:
    def __init__(self, state_dim, action_dim, lr_actor, lr_critic, gamma, K_epochs, eps_clip, action_std_init=0.6):

        self.gamma = gamma
        self.eps_clip = eps_clip
        self.K_epochs = K_epochs
        
        self.buffer = RolloutBuffer()

        self.policy = ActorCritic(state_dim, action_dim, action_std_init).to(device)
        self.optimizer = torch.optim.AdamW([
                        {'params': self.policy.actor.parameters(), 'lr': lr_actor},
                        {'params': self.policy.critic.parameters(), 'lr': lr_critic}
                    ])

        self.policy_old = ActorCritic(state_dim, action_dim, action_std_init).to(device)
        self.policy_old.load_state_dict(self.policy.state_dict())
        
        self.MseLoss = nn.MSELoss()

    def select_action(self, state):

          with torch.no_grad():
              state = torch.FloatTensor(state).to(device)
              action, action_logprob = self.policy_old.act(state)
            
          self.buffer.states.append(state)
          self.buffer.actions.append(action)
          self.buffer.logprobs.append(action_logprob)

          return action.item()


    def update(self):

        # Monte Carlo estimate of returns
        rewards = []
        discounted_reward = 0
        for reward, is_terminal in zip(reversed(self.buffer.rewards), reversed(self.buffer.is_terminals)):
            if is_terminal:
                discounted_reward = 0
            discounted_reward = reward + (self.gamma * discounted_reward)
            rewards.insert(0, discounted_reward)
            
        # Normalizing the rewards
        rewards = torch.tensor(rewards, dtype=torch.float32).to(device)
        rewards = (rewards - rewards.mean()) / (rewards.std() + 1e-7)

        # convert list to tensor
        old_states = torch.squeeze(torch.stack(self.buffer.states, dim=0)).detach().to(device)
        old_actions = torch.squeeze(torch.stack(self.buffer.actions, dim=0)).detach().to(device)
        old_logprobs = torch.squeeze(torch.stack(self.buffer.logprobs, dim=0)).detach().to(device)

        
        # Optimize policy for K epochs
        for _ in range(self.K_epochs):

            #logprobs, state_values, dist_entropy =  np.vectorize(self.policy.evaluate)(old_states, old_actions)

            # Evaluating old actions and values    
            logprobs, state_values, dist_entropy = self.policy.evaluate(old_states, old_actions)


            # match state_values tensor dimensions with rewards tensor
            state_values = torch.squeeze(state_values)
            
            # Finding the ratio (pi_theta / pi_theta__old)
            ratios = torch.exp(logprobs - old_logprobs.detach())

            # Finding Surrogate Loss
            advantages = rewards - state_values.detach()   
            surr1 = ratios * advantages
            surr2 = torch.clamp(ratios, 1-self.eps_clip, 1+self.eps_clip) * advantages

            # final loss of clipped objective PPO
            #loss = -torch.min(surr1, surr2) + 0.5*self.MseLoss(state_values, rewards) - 0.01*dist_entropy #0.5, 0.01
            loss = -torch.min(surr1, surr2)+ 0.5*self.MseLoss(state_values, rewards)- 0.01*dist_entropy
            
            # take gradient step
            self.optimizer.zero_grad()
            loss.mean().backward()
            self.optimizer.step()
            
        # Copy new weights into old policy
        self.policy_old.load_state_dict(self.policy.state_dict())

        # clear buffer
        self.buffer.clear()
    
    
    def save(self, checkpoint_path):
        torch.save(self.policy_old.state_dict(), checkpoint_path)
   

    def load(self, checkpoint_path):
        self.policy_old.load_state_dict(torch.load(checkpoint_path, map_location=lambda storage, loc: storage))
        self.policy.load_state_dict(torch.load(checkpoint_path, map_location=lambda storage, loc: storage))

################################### Training ###################################


####### initialize environment hyperparameters ######

env_name = "short path"
has_continuous_action_space = False

max_ep_len = 400                  # max timesteps in one episode 400
max_training_timesteps = int(1e5)   # break training loop if timeteps > max_training_timesteps

print_freq = max_ep_len * 4     # print avg reward in the interval (in num timesteps)
log_freq = max_ep_len * 2       # log avg reward in the interval (in num timesteps)
save_model_freq = int(2e4)      # save model frequency (in num timesteps)

action_std = None

################ PPO hyperparameters ################

update_timestep = max_ep_len* 4       # update policy every n timesteps* 4
K_epochs = 40               # update policy for K epochs
eps_clip = 0.2              # clip parameter for PPO, 0.2
gamma = 0.99                # discount factor 0.99

lr_actor = 0.0003     # learning rate for actor network 0.0003
lr_critic = 0.001     # learning rate for critic network 0.001 1 performs good
#lr_actor = 0.0001;lr_critic = 0.005
random_seed = 0         # set random seed if required (0 = no random seed)

#####################################################


print("training environment name : " + env_name)

# state space dimension
state_dim = 1

# action space dimension
action_dim = 18

###################### logging ######################

#### log files for multiple runs are NOT overwritten

log_dir = "PPO_logs"
if not os.path.exists(log_dir):
      os.makedirs(log_dir)

log_dir = log_dir + '/' + env_name + '/'
if not os.path.exists(log_dir):
      os.makedirs(log_dir)


#### get number of log files in log directory
run_num = 0
current_num_files = next(os.walk(log_dir))[2]
run_num = len(current_num_files)


#### create new log file for each run 
log_f_name = log_dir + '/PPO_' + env_name + "_log_" + str(run_num) + ".csv"

print("current logging run number for " + env_name + " : ", run_num)
print("logging at : " + log_f_name)

#####################################################

################### checkpointing ###################

run_num_pretrained = 0      #### change this to prevent overwriting weights in same env_name folder

directory = "PPO_preTrained"
if not os.path.exists(directory):
      os.makedirs(directory)

directory = directory + '/' + env_name + '/'
if not os.path.exists(directory):
      os.makedirs(directory)


checkpoint_path = directory + "PPO_{}_{}_{}.pth".format(env_name, random_seed, run_num_pretrained)
print("save checkpoint path : " + checkpoint_path)

def state_determin(action):
  new_state=np.array([action])

  return new_state

def reward_determine(old_state,state):
  
  end_bonus=0
  old_terminal= old_state[0]%6
  old_mode= int(old_state[0]/6)
  terminal= state[0]%6
  mode= int(state[0]/6)

  time_matrix=np.zeros([3,6,6])
  time_matrix[0,0,2] = 1
  time_matrix[0,0,1] = 2
  time_matrix[1,2,4] = 1
  time_matrix[1,1,3] = 2
  time_matrix[2,4,5] = 1
  time_matrix[2,3,5] = 2

  i=mode
  j=old_terminal
  k=terminal

  if time_matrix[i,j,k]==0:
    penalty = 100
  else:
    penalty = 0
  if state[0]%6 == 5 and old_state[0]!= 0:
      end_bonus=1000

  reward=-time_matrix[i,j,k]-penalty+end_bonus
  return reward

'''
def travel_time_determine(old_state,state):
    time_matrix = np.array([[0,1,2,0,0,0],
                      [2,0,0,1,2,0],
                      [2,0,0,2,2,0],
                      [0,2,2,0,0,1],
                      [0,2,2,0,0,2],
                      [0,0,0,2,2,0]])
    i=old_state[0]
    j=state[0]
    travel_time = time_matrix[i,j]
    return travel_time
'''

def done_determine(state):
  if state[0]%6 ==5 :
    done=1
  else:
    done=0
  return done

################# training procedure ################

# initialize a PPO agent
ppo_agent = PPO(state_dim, action_dim, lr_actor, lr_critic, gamma, K_epochs, eps_clip, action_std)

# track total training time
start_time = datetime.now().replace(microsecond=0)
print("Started training at (GMT) : ", start_time)

print("============================================================================================")

# logging file
log_f = open(log_f_name,"w+")
log_f.write('episode,timestep,reward\n')


# printing and logging variables
print_running_reward = 0
print_running_episodes = 0

log_running_reward = 0
log_running_episodes = 0

time_step = 0
i_episode = 0

#state_sample=[0]

# training loop
while time_step <= max_training_timesteps:

    #print('new round')
    state = np.array([0]) #起点  terminal 0
    current_ep_reward = 0
    #arrival_t=0

    for t in range(1, max_ep_len+1):
        # exmaine the activation of states
        #if  t >= arrival_t:
          # select action with policy
          #print('before',state)

        

          action = ppo_agent.select_action(state)
          previous_state=state
          state = state_determin(action)
          #print('after',state)
          
          

          reward = reward_determine(previous_state,state)#It is a floating data type value.
          is_over=done_determine(state)          #if 到达终点 done=1
          #route_time=travel_time_determine(previous_state,state)
          #arrival_t = t + route_time

          #state_sample.append(reward)# added
          '''
          # update activation
          if t >= arrival_t:
              activation = 1
          else:
              activation = 0
           '''
          # saving reward and is_terminals
          ppo_agent.buffer.rewards.append(reward)
          ppo_agent.buffer.is_terminals.append(is_over)
          
          time_step +=1
          current_ep_reward += reward
          
          # update PPO agent
          if time_step % update_timestep == 0:
              ppo_agent.update()
              
          # log in logging file
          if time_step % log_freq == 0:

              # log average reward till last episode
              log_avg_reward = log_running_reward / log_running_episodes
              log_avg_reward = np.round(log_avg_reward, 4)

              log_f.write('{},{},{}\n'.format(i_episode, time_step, log_avg_reward))
              log_f.flush()

              log_running_reward = 0
              log_running_episodes = 0

          # printing average reward
          if time_step % print_freq == 0:
              

              # print average reward till last episode
              print_avg_reward = print_running_reward / print_running_episodes
              print_avg_reward = np.round(print_avg_reward, 2)

              print("Episode : {} \t\t Timestep : {} \t\t Average Reward : {}".format(i_episode, time_step, print_avg_reward))

              print_running_reward = 0
              print_running_episodes = 0

              #print(state_sample) #added
              #state_sample = [0]
                     
          # save model weights
          if time_step % save_model_freq == 0:
              print("--------------------------------------------------------------------------------------------")
              print("saving model at : " + checkpoint_path)
              ppo_agent.save(checkpoint_path)
              print("model saved")
              print("Elapsed Time  : ", datetime.now().replace(microsecond=0) - start_time)
              print("--------------------------------------------------------------------------------------------")
            
          # break; if the episode is over
          if is_over==1:
              break


    print_running_reward += current_ep_reward
    print_running_episodes += 1

    log_running_reward += current_ep_reward
    log_running_episodes += 1

    i_episode += 1


log_f.close()


# print total training time
print("============================================================================================")
end_time = datetime.now().replace(microsecond=0)
print("Started training at (GMT) : ", start_time)
print("Finished training at (GMT) : ", end_time)
print("Total training time  : ", end_time - start_time)
print("============================================================================================")


Device set to : cpu
training environment name : short path
current logging run number for short path :  7
logging at : PPO_logs/short path//PPO_short path_log_7.csv
save checkpoint path : PPO_preTrained/short path/PPO_short path_0_0.pth
Started training at (GMT) :  2022-09-22 17:10:34
Episode : 318 		 Timestep : 1600 		 Average Reward : 204.89
Episode : 733 		 Timestep : 3200 		 Average Reward : 468.43
Episode : 1254 		 Timestep : 4800 		 Average Reward : 571.51
Episode : 1892 		 Timestep : 6400 		 Average Reward : 668.03
Episode : 2598 		 Timestep : 8000 		 Average Reward : 725.65
Episode : 3305 		 Timestep : 9600 		 Average Reward : 764.4
Episode : 4032 		 Timestep : 11200 		 Average Reward : 798.66
Episode : 4758 		 Timestep : 12800 		 Average Reward : 821.06
Episode : 5503 		 Timestep : 14400 		 Average Reward : 841.09
Episode : 6253 		 Timestep : 16000 		 Average Reward : 866.38
Episode : 7014 		 Timestep : 17600 		 Average Reward : 883.0
Episode : 7775 		 Timestep : 19200 		 Aver